In [1]:
import os

In [2]:
os.chdir("../")

In [20]:
from sklearn.metrics import silhouette_score
import dagshub
import mlflow
from urllib.parse import urlparse
from pathlib import Path
from src.Customer_segementation.utils.common import load_object

In [21]:
import pandas as pd
from dataclasses import dataclass

In [22]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    data: Path
    model_path: Path
    dim_red_model: Path


In [23]:
from src.Customer_segementation.constant import *
from src.Customer_segementation.utils.common import read_yaml, create_directories

In [24]:
class configurationManager: 
    def __init__(self,config_file_path=CONFIG_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.schema=read_yaml(schema_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self):
        config=self.config.model_evaluation

        model_evaluation_config=ModelEvaluationConfig(
            data=config.train_data_scaled,
            model_path=config.model_path,
            dim_red_model=config.dim_red_model

        )
        return model_evaluation_config

In [25]:
class ModelEvaluation:
    def __init__(self, config=ModelEvaluationConfig):
        self.config=config

    def evluation_matrix_for_mlflow(self, model, train_data): 
        dagshub.init(repo_owner='Abhikkumar619', repo_name='Customer_segmentation_project', mlflow=True)
        mlflow.set_registry_uri('https://dagshub.com/Abhikkumar619/Customer_segmentation_project.mlflow')
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store)


        with mlflow.start_run():
            prediction=model.predict(train_data)
            sil_score=silhouette_score(train_data ,model.labels_)
            mlflow.log_metric("silhouette_score",sil_score)

            if tracking_url_type_store !='file': 
                    mlflow.sklearn.log_model(model, "model", registered_model_name="ml_model")
            else: 
                mlflow.sklearn.log_model(model, "model")


    def inititate_model_evaluation(self):
        try: 
            # Load model from artifacts.
            model=load_object(Path(self.config.model_path))
            dim_model=load_object(Path(self.config.dim_red_model))
            
            x=pd.read_csv(self.config.data)

            dim_train=dim_model.transform(x)

            self.evluation_matrix_for_mlflow(model, dim_train)

           
        except Exception as e: 
            raise



In [27]:
try: 
    configmanager=configurationManager()
    evaluation_config=configmanager.get_model_evaluation_config()
    model_evaluation=ModelEvaluation(evaluation_config)
    model_evaluation.inititate_model_evaluation()
    
except Exception as e:
    
    raise e

[2024-07-01 06:57:48,008 : INFO : common : Yaml file read config/config.yaml successfully]
[2024-07-01 06:57:48,011 : INFO : common : Yaml file read schema.yaml successfully]
[2024-07-01 06:57:48,012 : INFO : common : Yaml file read params.yaml successfully]
[2024-07-01 06:57:48,012 : INFO : common : Directories created ['artifacts']]
[2024-07-01 06:57:48,013 : INFO : common : object load sucessfully from path: artifacts/model_trainer/best_Model.h5]
[2024-07-01 06:57:48,013 : INFO : common : object load sucessfully from path: artifacts/model_trainer/dim_red.h5]


Initialized MLflow to track repo "Abhikkumar619/Customer_segmentation_project"

[2024-07-01 06:57:48,638 : INFO : helpers : Initialized MLflow to track repo "Abhikkumar619/Customer_segmentation_project"]


Repository Abhikkumar619/Customer_segmentation_project initialized!

[2024-07-01 06:57:48,640 : INFO : helpers : Repository Abhikkumar619/Customer_segmentation_project initialized!]
https
[2024-07-01 06:58:07,369 : WARNING : connectionpool : Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /Abhikkumar619/Customer_segmentation_project.mlflow/api/2.0/mlflow/runs/create]


Registered model 'ml_model' already exists. Creating a new version of this model...
2024/07/01 06:58:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ml_model, version 2
Created version '2' of model 'ml_model'.
